# Import packages



In [ ]:
!pip install opencv-python

import pandas as pd
import numpy as np
import cv2
import os


from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

# Load data and pre-process images using CV library: Downsize images to 200x200, convert into tensor





In [ ]:
def corp_margin(img2):
    img2=np.asarray(img2)
    (row, col) = img2.shape
    row_top = 0
    raw_down = 0
    col_top = 0
    col_down = 0
    axis1=img2.sum(axis=1)
    axis0=img2.sum(axis=0)
    for r in range(0, row):
        if axis1[r] > 30:
            row_top = r
            break
    for r in range(row - 1, 0, -1):
        if axis1[r] > 30:
            raw_down = r
            break
    for c in range(0, col):
        if axis0[c] > 30:
            col_top = c
            break
    for c in range(col - 1, 0, -1):
        if axis0[c] > 30:
            col_down = c
            break
    a=raw_down+ 1 - row_top-(col_down+ 1-col_top)
    if a>0:
            w=raw_down+ 1-row_top
            col_down=int((col_top+col_down + 1)/2+w/2)
            col_top = col_down-w
            if col_top < 0:
                col_top = 0
                col_down = col_top + w
            elif col_down >= col:
                col_down = col - 1
                col_top = col_down - w
    else:
            w=col_down + 1- col_top
            raw_down = int((row_top + raw_down + 1) / 2 + w/2)
            row_top =  raw_down-w
            if row_top < 0:
                row_top = 0
                raw_down = row_top + w
            elif raw_down >= row:
                raw_down = row - 1
                row_top = raw_down - w
    if row_top==raw_down:
        row_top=0
        raw_down=99
        col_top = 0
        col_down = 99
    new_img = img2[row_top:raw_down + 1, col_top:col_down + 1]
    return new_img

In [ ]:
def read_ct_img_bydir(target_dir):
    img=cv2.imdecode(np.fromfile(target_dir,dtype=np.uint8),cv2.IMREAD_GRAYSCALE)
    img = corp_margin(img)
    img=cv2.resize(img,(200,200))
    return img

In [ ]:
def get_data():
    '''Loads raw data from image names of each class folder and assigns a label to it.'''
    
    target_dir1='../ <insert your path> /NiCT/'
    target_dir2='../ <insert your path> /data/pCT/'
    target_dir3='../ <insert your path> /data/nCT/'

    target_list1=[target_dir1+file for file in os.listdir(target_dir1)]
    target_list2=[target_dir2+file for file in os.listdir(target_dir2)]
    target_list3=[target_dir3+file for file in os.listdir(target_dir3)]

    target_list=target_list1+target_list2+target_list3

    # Assign labels: 0: 'non informative CT'
    #                1: 'positive Covid-19 CT'
    #                1: 'negative Covid-19 CT'
    y_list=to_categorical(np.concatenate(np.array([[0]*len(target_list1),
                                               [1]*len(target_list2),
                                               [2]*len(target_list3)])),3)

    X=np.array([read_ct_img_bydir(file) for file in target_list])[:,:,:,np.newaxis]

    return X, y_list

In [ ]:
X, y = get_data()
#np.shape(df)
#df[1045,:,:,:]